In [63]:
%matplotlib inline

In [64]:
import pandas as pd
import numpy as np

In [65]:
raw = 'raw'
df_train_all = pd.read_csv(f'{raw}/train.csv')
len(df_train_all)

891

In [66]:
from sklearn.model_selection import KFold

In [67]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

cl = GradientBoostingClassifier()
field_name = 'Survived'
feature_names = ['SibSp']

accuracies = []

kfold = KFold(random_state=42, n_splits=3)
for trn_idx, val_idx in kfold.split(df_train_all):
    df_train = df_train_all.iloc[trn_idx]
    df_val  = df_train_all.iloc[val_idx]

    y_train = df_train[field_name]
    y_val   = df_val[  field_name]

    del df_train[field_name], df_val[field_name]

    df_train = df_train[feature_names]
    df_val   = df_val[  feature_names]
    
    cl.fit(df_train, y_train)
    y_pred = cl.predict(df_val)

    acc = accuracy_score(y_val, y_pred)
    accuracies.append(acc)

accuracies

[0.6127946127946128, 0.569023569023569, 0.6464646464646465]

In [68]:
# Note to self: this looks handy
from sklearn.model_selection import cross_val_score
cross_val_score(cl, df_train_all[feature_names], df_train_all[field_name], cv=kfold)

array([0.61279461, 0.56902357, 0.64646465])

In [69]:
df_test = pd.read_csv(f'{raw}/test.csv')
y_test = cl.predict(df_test[feature_names])
df_submit = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': y_test})
df_submit.to_csv('submission.csv', index=False)

In [70]:
!ls

00. EDA.ipynb    10. CV.ipynb     raw              requirements.txt submission.csv


In [71]:
# this yields LB score of 0.61722 when submitted, looks reasonably close to CV results
np.round(np.mean(accuracies),4)

0.6094